In [11]:
# ! pip install -e /files/shared/ap/jupyter-notebook/pkgs/objFuncs
# ! pip install -e /files/shared/ap/jupyter-notebook/pkgs/pyBO

 # please re-start kernel if these packages are installed for the first time

In [12]:
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import datetime
import concurrent
from phantasy import caget,fetch_data

In [13]:
import objFuncs
from objFuncs import objFuncGoals
from objFuncs.util import plot_obj_history
from pyBO import pyBO

In [14]:
# increase budget a little more (+20?) if need +1~2% more beam
budget = 50
is_close_to_opt = True
if not is_close_to_opt:
    budget += 15

# check objFuncs machineIO, source and beam  

In [15]:
objFuncs._global_machineIO._test = False
objFuncs._global_machineIO._fetch_data_time_span = 4
objFuncs._global_machineIO._ensure_set_timewait_after_ramp : 0.25
objFuncs._global_machineIO.view()

   _test : False
   _ensure_set_timeout : 30
   _ensure_set_timewait_after_ramp : 0.25
   _fetch_data_time_span : 4
   _return_obj_var : False
   _check_chopper_blocking : True
   _n_popup_ramping_not_OK : 0
   _verbose : False


In [16]:
SCS = caget("ACS_DIAG:DEST:ACTIVE_ION_SOURCE")
ion = caget("FE_ISRC"+str(SCS)+":BEAM:ELMT_BOOK")
Q = caget("FE_ISRC"+str(SCS)+":BEAM:Q_BOOK")
A = caget("FE_ISRC"+str(SCS)+":BEAM:A_BOOK")
# AQ = caget("FE_ISRC2:BEAM:MOVRQ_BOOK")
AQ = A/Q
ion = str(A)+ion+str(Q)
print('SCS'+str(SCS), ion, 'A/Q=',AQ)

SCS2 18O7 A/Q= 2.5714285714285716


In [17]:
now0 = datetime.datetime.now()
now0str = now0.strftime('%Y%m%d_%H%M')
fname = now0str+'['+ion+'][pyBO_TR][FS2]trajectory'
fname

'20240210_1551[18O7][pyBO_TR][FS2]trajectory'

# preprare objective

In [18]:
decision_CSETs = [
                  'FS2_BTS:PSC2_D3962:I_CSET',
                  'FS2_BBS:PSD_D3979:I_CSET',
                  'FS2_BBS:PSD_D4034:I_CSET',
                  'FS2_BBS:PSD_D4072:I_CSET',
                  'FS2_BBS:PSD_D4127:I_CSET',
                  ]
decision_min = []
decision_max = []
decision_tols = []

for pv in decision_CSETs:
    x0 = caget(pv)
    if x0 is None:
        raise ValueError(f'pv: {pv} is not connected')
    if "PSD" in pv:
        if x0<0.0:
            decision_min.append(x0*(1+0.002))
            decision_max.append(x0*(1-0.002))
        else:
            decision_min.append(x0*(1-0.002))
            decision_max.append(x0*(1+0.002))
        decision_tols.append(0.04)    
    elif "PSC" in pv:
        decision_min.append(-2.0)
        decision_max.append( 2.0)
        decision_tols.append(0.2)
    else:
        raise ValueError(f'cannot auto-determine bounds of pv: {pv}')
            
print(f'decision_min : {decision_min}')
print(f'decision_max : {decision_max}')
print(f'decision_tols: {decision_tols}')

decision_min : [-2.0, -113.4971812799999, -114.06222912000004, -114.10431311999997, -114.48707712000008]
decision_max : [2.0, -113.04409871999988, -113.60689088000004, -113.64880687999997, -114.03004288000007]
decision_tols: [0.3, 0.04, 0.04, 0.04, 0.04]


In [ ]:
obj = objFuncGoals(
    decision_CSETs=decision_CSETs,
    decision_min = decision_min,
    decision_max = decision_max,
    decision_tols= decision_tols,
    objective_goal   = {
                        'FS2_BBS:BPM_D4019:XPOS_RD': 0,
                        'FS2_BBS:BPM_D4054:XPOS_RD': 0,
                        'FS2_BBS:BPM_D4087:XPOS_RD': 0,
                        'FS2_BMS:BPM_D4142:XPOS_RD': 0,
                        'FS2_BMS:BPM_D4164:XPOS_RD': 0,
                        'FS2_BMS:BPM_D4177:XPOS_RD': 0,
                        },
    objective_weight = {
                        'FS2_BBS:BPM_D4019:XPOS_RD': 1.0,
                        'FS2_BBS:BPM_D4054:XPOS_RD': 1.0,
                        'FS2_BBS:BPM_D4087:XPOS_RD': 1.0,
                        'FS2_BMS:BPM_D4142:XPOS_RD': 1.0,
                        'FS2_BMS:BPM_D4164:XPOS_RD': 1.0,
                        'FS2_BMS:BPM_D4177:XPOS_RD': 1.0,
                        },
    objective_norm   = {
                        'FS2_BBS:BPM_D4019:XPOS_RD': 2,
                        'FS2_BBS:BPM_D4054:XPOS_RD': 2,
                        'FS2_BBS:BPM_D4087:XPOS_RD': 2,
                        'FS2_BMS:BPM_D4142:XPOS_RD': 1,
                        'FS2_BMS:BPM_D4164:XPOS_RD': 1,
                        'FS2_BMS:BPM_D4177:XPOS_RD': 1,
                        },
    objective_fill_none_by_init = True,
)

 # Prepare plot callbacks

In [ ]:
# define what to plot
plot_CSETs = plot_obj_history(
                obj.history['decision_CSETs'],
                title = 'decision_CSETs',
                inline = True,
                )
plot_RDs = plot_obj_history(
                obj.history['objective_RDs'],
                title = 'objective_RDs',
                inline = True,
                )
plot_objs = plot_obj_history(
            obj.history['objectives'],
            title = 'objectives',
            inline = True,
            )
callbacks = [plot_CSETs,plot_RDs,plot_objs]

# evaluate objective and plot
def obj_callbacks(x):
    return obj(x,callbacks=callbacks)

# Run TR-BO

In [ ]:
# run optimizer
bounds_diff = obj.decision_bounds[:,1] - obj.decision_bounds[:,0]
if is_close_to_opt:
    bounds = np.array(list(zip(obj.x0-0.075*bounds_diff, obj.x0+0.075*bounds_diff)))
    n_init = len(obj.x0)
else:
    bounds = obj.decision_bounds
    n_init = min(max(int(budget/4) , 2*len(obj.x0)) , budget)
    
print(bounds)
bo, X_pending, Y_pending_future = pyBO.runBO(
                                    obj_callbacks,  
                                    bounds = bounds,
                                    n_init = n_init,
                                    x0 = obj.x0,
                                    budget = n_init+1,
                                    batch_size=1,
                                    path="./log/",
                                    tag=fname+'_pyBO_history',
                                    write_log = False)
beta = 9
for i in range(budget-n_init-3):
    x_best,y_best = bo.best_sofar()
    # if all FC reading data is below 0.5 uA do the global search.
    if np.max(obj.history['objective_RDs']['values'])<0.5 and not is_close_to_opt:
        bounds = obj.decision_bounds
    else:
        bounds = np.array(list(zip(x_best-0.05*bounds_diff, x_best+0.05*bounds_diff)))
        beta *= 0.9
    print(bounds)    
    acquisition_func_args = {'beta':max(beta,1)}
    X_pending, Y_pending_future= bo.loop( 
                                    n_loop=1,  # number of additional optimization interation
                                    func_obj = obj_callbacks,
                                    bounds = bounds,
                                    acquisition_func_args = acquisition_func_args,
                                    X_pending = X_pending, 
                                    Y_pending_future = Y_pending_future,
                                    batch_size = 1,
                                    write_log = False,
                                    )

for i in range(2):
    x_best,y_best = bo.best_sofar()
    bounds = np.array(list(zip(x_best-0.05*bounds_diff, x_best+0.05*bounds_diff)))
    acquisition_func_args = {'beta':0.01}
    X_pending, Y_pending_future= bo.loop( 
                                    n_loop=1,  # number of additional optimization interation
                                    func_obj = obj_callbacks,
                                    bounds = bounds,
                                    acquisition_func_args = acquisition_func_args,
                                    X_pending = X_pending, 
                                    Y_pending_future = Y_pending_future,
                                    batch_size = 1,
                                    write_log = False,
                                    polarity_change_time = 0,
                                    )
for f in callbacks:
    f.close()

In [ ]:
ax = bo.plot_obj_history()

In [ ]:
# set to best solution 
x_best,y_best_old = bo.best_sofar()
y_best_new = obj(x_best)
print(x_best,y_best_old[0],y_best_new)   # check if best solution objective value is consistent

In [ ]:
if not objFuncs._global_machineIO._test:
    obj.write_log(fname=os.path.join('/files/shared/ap/jupyter-notebook/data/objFuncs_log/',fname))

In [ ]:
now1 = datetime.datetime.now()
now1str = str(now1)[:str(now1).rfind(':')].replace(' ','_').replace(':','').replace('-','')
time_delta = now1 - now0
print("time took:",str(time_delta.total_seconds()/60)[:4],'min')

# Visualize Surrogate model

In [ ]:
# fixed_values_for_each_dim = {2:x_best[2],3:x_best[3]}  # fix values to visualize high dim surrogate model
fixed_values_for_each_dim = None                         # do not fix values but project maximum. Can take long time to plot
batch_size = 1

In [ ]:
# # plot acuquisition functions for the first 4 epochs. Can take long time for decision dim >= 4
# for epoch in range(4):   
#     fig,ax = plt.subplots(1,batch_size,figsize=(4*batch_size,3))
#     if batch_size ==1:
#         ax = [ax]
#     for i in range(batch_size):
#         bo.plot_acquisition_2D_projection(project_maximum=True,
#                                           epoch=epoch,
#                                           i_query=i,
#                                           grid_ponits_each_dim = 16,
#                                           fixed_values_for_each_dim=fixed_values_for_each_dim,
#                                           fig=fig,ax=ax[i],);
#         ax[i].legend()
#         ax[i].set_xlabel(obj.decision_CSETs[0])
#         ax[i].set_ylabel(obj.decision_CSETs[1])
#     fig.tight_layout()

In [ ]:
# # plot acuquisition functions for the last 4 epochs. Can take long time for decision dim >= 4
# for epoch in range(-6,-2):
#     fig,ax = plt.subplots(1,batch_size,figsize=(4*batch_size,3))
#     if batch_size ==1:
#         ax = [ax]
#     for i in range(batch_size):
#         bo.plot_acquisition_2D_projection(project_maximum=True,
#                                           epoch=epoch,
#                                           i_query=i,
#                                           grid_ponits_each_dim = 16,
#                                           fixed_values_for_each_dim=fixed_values_for_each_dim,
#                                           fig=fig,ax=ax[i],);
#         ax[i].legend()
#         ax[i].set_xlabel(obj.decision_CSETs[0])
#         ax[i].set_ylabel(obj.decision_CSETs[1])
#     fig.tight_layout()

In [ ]:
# plot surrogate mean model of the last epoch. Can take long time for decision dim >= 4

t0 = time.monotonic()
from math import ceil
nplot = int(0.5*len(obj.decision_CSETs))
nrow = ceil(0.5*nplot)
fig,ax = plt.subplots(nrow,2,figsize=(8,3*nrow))
for i in range(nrow):
    for j in range(2):
        n = 2*i+j
        if nrow>1:
            ax_ = ax[i,j]
        else:
            ax_ = ax[j]
        if n >= nplot:
            ax_.set_visible(False)
            break
        bo.plot_model_2D_projection(project_maximum=True,
                                    dim_xaxis = 2*n,
                                    dim_yaxis = 2*n+1,
                                    grid_ponits_each_dim = 16,
                                    fixed_values_for_each_dim=fixed_values_for_each_dim,
                                            fig=fig,ax=ax_);
        ax_.set_xlabel(obj.decision_CSETs[2*n  ])
        ax_.set_ylabel(obj.decision_CSETs[2*n+1])
        ax_.legend()
fig.tight_layout()

    
print('time took: ',time.monotonic()-t0)